<a href="https://colab.research.google.com/github/attentionmech/tensorlens/blob/main/tensorlens/notebooks/gpt2_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -q uv
!uv run --prerelease=allow https://raw.githubusercontent.com/attentionmech/smolbox/refs/heads/main/smolbox/tools/inspect/tensorlens_activations.py run --notebook True

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
True


In [29]:
import fire
import torch
from smolbox.core.state_manager import AUTORESOLVE, resolve
from smolbox.core.tool_manager import BaseTool
from transformers import AutoTokenizer, AutoModelForCausalLM

from tensorlens.tensorlens import trace, viewer

class TensorLensActivations(BaseTool):
    def __init__(
        self,
        model_path=AUTORESOLVE,
        prompt="Once upon a time "*10,
        max_new_tokens=1,  # Generate only 1 new token
        host="localhost",
        port=8000,
        notebook=False,
    ):
        self.model_path = resolve("model_path", model_path)
        self.text_input = prompt
        self.max_new_tokens = int(max_new_tokens)
        self.host = host
        self.port = port
        self.notebook = notebook

    def run(self):
        # Load tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(self.model_path)
        model = AutoModelForCausalLM.from_pretrained(self.model_path)
        model.eval()

        # Set padding token if it's None
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        if model.config.pad_token_id is None:
            model.config.pad_token_id = tokenizer.pad_token_id

        # Tokenize the input text
        input_ids = tokenizer(
            self.text_input or "Once upon a time"*10,
            return_tensors="pt"
        ).input_ids

        # Step 1: Perform a forward pass to get activations for the prompt
        with torch.no_grad():
            outputs = model(input_ids=input_ids, output_hidden_states=True)

        # Trace activations for the prompt tokens (only)
        for step_idx, layer_outputs in enumerate(outputs.hidden_states):
            for layer_idx, hidden in enumerate(layer_outputs):
                trace(f"layer_{layer_idx}_step_{step_idx}", hidden.detach().cpu().numpy())

        # Step 2: Generate 1 new token based on the prompt
        with torch.no_grad():
            outputs_gen = model.generate(
                input_ids=input_ids,
                max_new_tokens=self.max_new_tokens,  # Only generate 1 new token
                return_dict_in_generate=True,
                output_hidden_states=True,
                output_scores=False,
                output_attentions=False,
                do_sample=False,  # Greedy decoding
                pad_token_id=tokenizer.pad_token_id,
            )

        # Step 3: Trace activations for the first generated token
        for step_idx, layer_outputs in enumerate(outputs_gen.hidden_states):
            for layer_idx, hidden in enumerate(layer_outputs):
                trace(f"layer_{layer_idx}_step_gen_{step_idx}", hidden.detach().cpu().numpy())

        viewer(height="600",width='100%', port=self.port, host=self.host, notebook=self.notebook)
        return True


In [30]:
TensorLensActivations(notebook=True).run()

<IPython.core.display.Javascript object>

True